In [ ]:
1

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os
import sys

sys.path.append("../technique")
import pandas as pd
import numpy as np

# CCS

## Base

In [ ]:
raw_df = pd.read_csv(
    f"{os.getenv('DATA_FOLDER')}culture/CCS_ELEVES_FULL.csv",
    delimiter=";",
    encoding="windows-1250",
    index_col=0,
    decimal=",",
)
version = 2
raw_df = pd.read_excel(
    f"{os.getenv('DATA_FOLDER')}culture/ccs/Données Tarification solidaire Eléments familles Saison 2022-2023 simulation.xlsx",
    sheet_name="DuoNET-Export_Familles_25_09_20",
)
version = 3

In [ ]:
insee_df = pd.read_csv(
    f"{os.getenv('DATA_FOLDER')}culture/ccs/2023-10-10_CCS_QFMES.csv",
    delimiter=";",
    encoding="windows-1250",
    index_col=0,
    decimal=",",
)
version = 4

In [ ]:
comp_df = pd.DataFrame(
    {
        "caf": raw_df.groupby("LABEL")["Montant.facturé"].count(),
        "insee": (insee_df.groupby("LABEL")["Montant.facturé"].count() * 0.1),
    }
)
comp_df["bug"] = comp_df.insee > comp_df.caf

In [ ]:
comp_df[comp_df.bug]

In [ ]:
raw_df[["Montant.facturé", "LABEL"]].to_pickle(
    f"{os.getenv('DATA_FOLDER')}minimales/CCS_ELEVES_anon_base_v{version}.pickle"
)

In [ ]:
insee_df.rename(columns={"SERIE": "sample_id"})[
    ["Montant.facturé", "LABEL", "sample_id", "QFEMS"]].to_pickle(
    f"{os.getenv('DATA_FOLDER')}minimales/CCS_ELEVES_anon_insee_v{version}.pickle"
)

# CRR

In [ ]:
filename = "Copie de Copie de Elèves 2022-23 - dominantes + montant facturé + tranche HA v2.xlsx"
version = 1
filename = "Copie de Copie de Copie de Elèves 2022-23 - dominantes + montant facturé + tranche HA.xlsx"
version = 2
filename = "0925 Elèves 2022-23 - dominantes + montant facturé + tranche HA.xlsx"
version = 3
filename = (
    "Copie de 0925 Elèves 2022-23 - dominantes + montant facturé + tranche HA.xlsx"
)
version = 4
version = 5
df_cours_crr_base = pd.read_excel(
    f"{os.getenv('DATA_FOLDER')}culture/Conservatoire/{filename}"
)

source = "base"

In [ ]:
df_cours_crr_base['ID'] = df_cours_crr_base.index.values
df_cours_crr_base.rename(columns={"habitant EMS":"habitant.EMS"}, inplace=True)

## INSEE

In [ ]:
df_crr_bourses_raw = pd.read_excel(
    f"{os.getenv('DATA_FOLDER')}culture/Conservatoire/PV commission d'attribution bourses 2022-23.xlsx",
    skiprows=1,
)
df_crr_bourses_raw_subset = df_crr_bourses_raw[
    (~df_crr_bourses_raw.NIVEAU.isna())
    * (~df_crr_bourses_raw["Montant Bourses 2022/2023"].isna())
]
assert (
    df_crr_bourses_raw_subset.merge(
        df_crr_bourses_raw_subset, on=["NOM", "PRENOM"]
    ).shape[0]
    == df_crr_bourses_raw_subset.shape[0]
)

def build(df_cours_crr, source="base"):
    raw_columns = [
            "index",
            "NOM",
            "PRENOM",
            "N°",
            "MontantFactureSurEleve",
            "Montant Bourses 2022/2023",
        ]
    
    if source == "insee":
        raw_columns.insert(0, "sample_id")
    bourse_data = df_cours_crr.merge(
        df_crr_bourses_raw_subset,
        how="right",
        left_on=["Nom", "Prenom"],
        right_on=["NOM", "PRENOM"],
    )[raw_columns].rename(
        columns={
            "MontantFactureSurEleve": "Tarif",
            "Montant Bourses 2022/2023": "Bourse",
            "NOM": "Nom",
            "PRENOM": "Prenom",
        }
    )
    
    if source == "insee":
        sample_count = 10
        assert (bourse_data[bourse_data.Tarif.isna()]['N°'] == [64]).all()
    else:
        sample_count = 1
        assert bourse_data[bourse_data.Tarif.isna()].shape[0] == 0
    bourse_rows = bourse_data.groupby("N°").first()
    bourse_rows["TotalPayé"] = bourse_data.groupby("N°").Tarif.sum() / sample_count
    bourses = bourse_data.merge(
        bourse_rows[["Nom", "Prenom", "TotalPayé"]], on=["Nom", "Prenom"]
    )
    bourses["Proportion"] = bourses.Tarif / bourses.TotalPayé
    bourses["strasbourg_conservatoire_bourse_historique"] = (
        bourses.Proportion * bourses.Bourse
    )

    if source == "insee":
        assert (
            bourses.groupby("index").strasbourg_conservatoire_bourse_historique.count().max()
            == sample_count
        )

    columns = [
        "index",
        "MontantFactureSurEleve",
        "service",
        "Cycle.1",
        "agent",
        "Tranche",
        "habitant.EMS",
    ]
    
    base_columns = ["index", "strasbourg_conservatoire_bourse_historique"]
    join_columns = ['index']
    if source == "insee":
        columns.append("sample_id")
        columns.append("QFEMS")
        base_columns.insert(0, "sample_id")
        join_columns.insert(0, "sample_id")

    df_crr = df_cours_crr[columns].merge(
        bourses[base_columns],
        on=join_columns,
        how="left",
    )
    df_crr.strasbourg_conservatoire_bourse_historique.fillna(0, inplace=True)
    df_crr.strasbourg_conservatoire_bourse_historique.sum()
    if source == "insee":
        assert (
            df_crr.groupby("sample_id").strasbourg_conservatoire_bourse_historique.sum()
            == 14800
        ).all()
    else:
        assert (
            df_crr.strasbourg_conservatoire_bourse_historique.sum()
            == 15000
        ).all()

    return df_crr

In [ ]:
version = 6

In [ ]:
crr_base = build(df_cours_crr_base)
crr_base.to_excel(
    f"{os.getenv('DATA_FOLDER')}minimales/conservatoire_{source}_v{version}.xlsx",
    index=False,
)

In [ ]:
insee_df = pd.read_csv(
    f"{os.getenv('DATA_FOLDER')}culture/Conservatoire/2023-10-10_CRR_QFMES.csv",
    delimiter=";",
    encoding="windows-1250",
    index_col=0,
    decimal=",",
)
source = "insee"
insee_df.rename(columns={"SERIE": "sample_id"}, inplace=True)

In [ ]:
sample_count = (insee_df.ID == insee_df.ID.iloc[0]).sum()
count = len(insee_df) / sample_count
insee_df["sample_id"] = np.repeat(list(range(sample_count)), count)

In [ ]:
crr_insee = build(insee_df, "insee")
crr_insee.to_excel(
    f"{os.getenv('DATA_FOLDER')}minimales/conservatoire_{source}_v{version}.xlsx",
    index=False,
)